In [ ]:
import numpy as np
import pandas as pd
import cmocean as cmo
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from windrose import WindroseAxes

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['mathtext.fontset'] = 'cm'

# BOM warns of more wet weather, dangerous conditions as east coast low looms off NSW

<img src="https://live-production.wcms.abc-cdn.net.au/846da2c9e8fa6b4b89e28591957cb8f9?impolicy=wcms_crop_resize&cropH=1123&cropW=1996&xPos=25&yPos=0&width=862&height=485" width="100%">

_Moist air streaming in from the east as the low looms offshore on the 02/03/22 
source:_ [Earth Nullschool](https://earth.nullschool.net)


## What is an east coast low?

In [ ]:
from IPython.display import YouTubeVideo

def display_yotube_video(url, **kwargs):
    """
    Displays a Youtube video in a Jupyter notebook.
    
    Args:
        url (string): a link to a Youtube video.
        **kwargs: further arguments for IPython.display.YouTubeVideo
    
    Returns:
        YouTubeVideo: a video that is displayed in your notebook.
    """
    id_ = url.split("=")[-1]
    return YouTubeVideo(id_, **kwargs)

display_yotube_video("https://www.youtube.com/watch?v=FCgULsMi9iQ", width=560, height=315)

We will look at the evolution of the wave field offshore Sydney based on the dataset from the NSW Ocean Wave Data Collection Program operated by the Manly Hydraulics Laboratory.

I did already saved the dataset for the relevant dates from here: https://mhl.nsw.gov.au/Station-SYDDOW.

### Dataset modifications

The link above allows for the wave height, period and direction to be obtained separately as 3 `csv` files.

From these files, we have access to the following variables:

+ `Hs`: significant wave height
+ `Hmax`: maximum wave height
+ `dir`: wave direction from North
+ `Tp`: Peak period 
+ `Tz`: zero crossing, the average period of the highest 1/3 waves, ie `Hs`.

We will combine all these variables in a unique dataframe:

In [ ]:
hs1 = pd.read_csv('../../dataset/Hs_ECL22.csv')
dir1 = pd.read_csv('../../dataset/dir_ECL22.csv')
T1 = pd.read_csv('../../dataset/T_ECL22.csv')

# initialize data of lists.
data = {'Date':hs1['Date'],
        'Hmax':hs1['Sydney (Hmax)'],
        'Hs':hs1['Sydney (Hs)'],
        'dir':dir1['Sydney (Wave Direction)'],
        'Tz':T1['Sydney (Tz)'],
        'TP1':T1['Sydney (TP1)'],
       }

# Create the pandas DataFrame
df = pd.DataFrame(data)
df.Date = pd.to_datetime(df['Date'], format='%Y-%m-%d %H:%M:%S.%f')
df.set_index(['Date'],inplace=True)
df

### Plotting the buoy record

In [ ]:
fig = plt.figure(figsize=(12,5), dpi=60) 
plt.plot(df.Hs, linestyle = 'solid', c='b', label='Hs')
plt.plot(df.Hmax, linestyle = 'solid', c='r', label='Hmax')
plt.legend(loc=0, frameon=False)
plt.ylabel("Wave height (m)")
plt.xlabel("Dates")
plt.show()

fig = plt.figure(figsize=(12,5), dpi=60) 
plt.plot(df.Tz, linestyle = 'solid', c='orange', label='Tz')
plt.plot(df.TP1, linestyle = 'solid', c='g', label='Tp')
plt.legend(loc=0, frameon=False)
plt.ylabel("Wave period (s)")
plt.xlabel("Dates")
plt.show()

fig = plt.figure(figsize=(12,5), dpi=60) 
plt.plot(df.dir, linestyle = 'solid', c='purple', label='dir')
plt.legend(loc=0, frameon=False)
plt.ylabel("Wave direction (degrees)")
plt.xlabel("Dates")
plt.show()

### Peak wave period

The peak wave period, `Tp`, is the wave period with the highest energy. 

The analysis of the distribution of the wave energy as a function of wave frequency f=1/T for a time-series of individual waves is referred to as a spectral analysis. 

Wind wave periods (frequencies) often follow the so-called **JONSWAP** or **Pierson-Moskowitz** spectra. The peak wave period is extracted from the spectra. As a rule of thumb the following relation can be used:

$$ T_p \simeq 5. \sqrt{H_{m}} $$

Let see how this relationship work for the offshore Sydney buoy dataset we have just downloaded:

In [ ]:
# We compute Tp from the above relationship 
hWave = np.arange(0,10,0.1)
Tp = 5.*np.sqrt(hWave)

Now we plot our dataset and the calculated `Tp` on the same graph:

In [ ]:
fig = plt.figure(figsize=(10,4))
ax = plt.gca()
pts = plt.scatter(df.Hmax, df.TP1, c=df.dir.values, cmap=cmo.cm.deep,
           edgecolor='black',alpha=0.75, zorder=4)

ax.plot(hWave, Tp, linestyle='--', color = 'k', lw=1, zorder=2,label='$T_p=5.0\sqrt{H_s}$')
ax.set_ylabel('Wave period (s)',fontsize=10)
ax.set_xlabel('Wave height (m)',fontsize=10)

# Color bar
cbar = fig.colorbar(pts, ax=ax, fraction=0.027, pad=0.065)
cbar.set_label('Direction in degrees', labelpad=5, fontsize=10)
cbar.ax.tick_params(labelsize=8)

plt.legend(fontsize=10, loc=4, frameon=False)
plt.show()
fig.clear()
plt.close(fig)

### Directional distribution of the wave heights

This distribution is often presented in the form of a **wave rose**. Like this:

In [ ]:
fig = plt.figure(figsize=(5,5), dpi=60) 
wax = WindroseAxes.from_ax(fig=fig)
wax.bar(df.dir, df.Hs, bins=np.arange(0, 5, 1), edgecolor='w', cmap=cmo.cm.deep, normed=True)
wax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
wax.set_yticks(np.arange(10, 80, step=10))
wax.set_yticklabels(np.arange(10, 80, step=10))
wax.set_legend(loc='best', bbox_to_anchor=(1.2, 1.05))
plt.show()

or:

In [ ]:
fig = plt.figure(figsize=(5,5), dpi=60) 
wax = WindroseAxes.from_ax(fig=fig)
wax.box(df.dir, df.Hs, bins=np.arange(0, 5, 1), edgecolor='w', cmap=cmo.cm.deep, normed=True)
wax.set_xticklabels(['E', 'NE', 'N', 'NW',  'W', 'SW', 'S', 'SE'])
wax.set_yticks(np.arange(10, 80, step=10))
wax.set_yticklabels(np.arange(10, 80, step=10))
wax.set_legend(loc='best', bbox_to_anchor=(1.2, 1.05))
plt.show()